In [53]:
import pandas as pd
import numpy as np
import geopandas as gpd
import timeit
import scipy

from scipy import stats
from pandas import ExcelWriter

pd.set_option('display.max_columns', None)
idx = pd.IndexSlice

In [93]:
input_path = '../../data/input/dhis2/new_system/'
input_path_old = '../../data/input/dhis2/old_system/'

shapes_path = '../../data/shapes/district/districts_17_19_clean.shp'
facility_path = '../../data/input/hospitals/original_data/'
pop_path = '../../data/input/demographics/'

output_path = '../../data/output/sprint3_analysis/'

# Fetch my data


## Set up

In [55]:
# build a small fucntion to split the string column name of the data download as pivot 

def split(strng, sep, occ):
    strng = strng.split(sep)
    return sep.join(strng[occ[1]:]), sep.join(strng[:occ[0]]), sep.join(strng[occ[0]:occ[1]])[:3]

In [111]:
# Creating a dict of names to replace district names

district_name_dict = {'SEMBABULE': 'SSEMBABULE', 'MADI-OKOLLO': 'MADI OKOLLO', 'LUWEERO':'LUWERO'}

# For a fully automated one, will need to do fuzzy matching


In [57]:
# To clean the data downloaded in a pivot format

def get_clean_stack(df,drop):

    df['district']=df['orgunitlevel3'].apply(lambda x: x[:-9].upper())
    df['district'].replace(district_name_dict,inplace=True)
    
    df.set_index(['district','organisationunitid'],drop=True,inplace=True)
    
    cols = np.arange(0,9)
    df.drop(df.columns[cols],axis=1,inplace=True)
    
    cols = df.columns
    new_cols=[]
    for col in cols:
        new_cols.append(split(col,' ',[-2,-1]))
    df.columns=pd.MultiIndex.from_tuples(new_cols,names=['year','indic','month'])
    
    if drop != None:
        df.drop(drop,axis=1,inplace=True,level=2)
    
    df1=df.copy().stack(level=[0,1,2],dropna=False).reset_index()
    df1.rename(columns={0:'value'},inplace=True)
    
    return df1

In [58]:
# To clean the data downloaded in a slightly different pivot format

def get_clean_stack_report(df,drop):

    month_dict={'01':'Jan','02':'Feb','03':'Mar','04':'Apr',
                '05':'May','06':'Jun','07':'Jul','08':'Aug',
                '09':'Sep','10':'Oct','11':'Nov','12':'Dec'}
    
    df['district']=df['orgunitlevel3'].apply(lambda x: x[:-9].upper())
    df['district'].replace(district_name_dict,inplace=True)

    df['year']=df['periodcode'].astype('str').apply(lambda x: x[:4])
    df['month']=df['periodcode'].astype('str').apply(lambda x: x[-2:]).replace(month_dict)
    
    if drop != None:
        drop=list(drop)
        df=df[~df['month'].isin(drop)]
    
    df.set_index(['district','organisationunitid','year','month'],drop=True,inplace=True)
        
    cols = np.arange(0,12)
    df.drop(df.columns[cols],axis=1,inplace=True)
    
    df1=df.copy().stack(dropna=False).reset_index()
    df1.rename(columns={0:'value','level_4':'indic'},inplace=True)
    
    return df1

## New data


In [59]:
# to get all the new data files together
def fetch_new_data (filepaths,filepaths_no_dec,filepaths_report):
    df = pd.DataFrame(columns = ['district', 'organisationunitid', 'year', 'indic', 'month', 'value'])
    for x in filepaths:
        x_df=get_clean_stack(pd.read_csv(x),drop='Dec')
        df=pd.concat([df,x_df])
    for x in filepaths_no_dec:
        x_df=get_clean_stack(pd.read_csv(x),drop=None)
        df=pd.concat([df,x_df])
    for x in filepaths_report:
        x_df=get_clean_stack_report(pd.read_csv(x),drop='Dec')
        df=pd.concat([df,x_df])
    return df


In [60]:
# creating the new data stack

filepaths = [input_path+'new_epi_data_by_facility.csv',
            input_path+'new_mnch_data_by_facility.csv',
            input_path+'new_sam_data_by_facility.csv',
            input_path+'new_lbw_data_by_facility.csv',
            input_path+'new_vitamin_data_by_facility.csv']

filepaths_no_dec = [input_path+'may_new_sam_data_by_facility.csv',
                    input_path+'may_new_epi_data_by_facility.csv']

filepaths_report = [input_path+'new_reporting_by_facility.csv']

new_stack = fetch_new_data (filepaths=filepaths,filepaths_no_dec=filepaths_no_dec,filepaths_report=filepaths_report)

C:\Users\Dalberg\anaconda3\envs\icohs\lib\site-packages\IPython\core\interactiveshell.py:3254: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


## Old data

For now extremely messy, had to be done bit by bit in random order, so quite some cleaning needed Ill do here, than put all into one nice file

In [61]:
# to get all the old data files together
def fetch_old_data (filepaths,filepaths_report):
    df = pd.DataFrame(columns = ['district', 'organisationunitid', 'year', 'indic', 'month', 'value'])
    for x in filepaths:
        x_df=get_clean_stack(pd.read_csv(x),drop=None)
        df=pd.concat([df,x_df])
    for x in filepaths_report:
        x_df=get_clean_stack_report(pd.read_csv(x),drop=None)
        df=pd.concat([df,x_df])    
    return df

In [62]:

# creating the old data stack

# For now a bit all over the placedue to download limitations 

filepaths = [input_path_old+"/epi/EPI - BCG doses given.csv",
             input_path_old+"/epi/EPI - DPT-HepB-HIB 1 doses given.csv",
             input_path_old+"/epi/EPI - DPT-HepB-HIB 3 doses given.csv",
             input_path_old+"/epi/EPI - PCV 1 doses given.csv",
             input_path_old+"/epi/EPI - PCV 3 doses given.csv",
             input_path_old+"/epi/EPI - MR 1 doses given.csv",
             input_path_old+'/mat/admission_newborn.csv',
             input_path_old+'/mat/ANC1_ANC4.csv',
             input_path_old+'/mat/births.csv',
             input_path_old+'/sam/lbw.csv',
             input_path_old+'/sam/sam_mam.csv',
             input_path_old+'/sam/lbw_abs.csv',
             input_path_old+'/sam/vitamin.csv']

filepaths_report = [input_path_old+'old_reporting_by_facility.csv']

old_stack = fetch_old_data (filepaths=filepaths,filepaths_report=filepaths_report)

## Population data

In [175]:
pop = pd.read_csv(pop_path+'UBOS_pop_proj_2015-25.csv')

def get_clean_pop_stack(pop):
    pop['District']=pop['District'].apply(lambda x: x.upper())
    pop['District'].replace(district_name_dict,inplace=True)

    pop['age']= pop['Single Years'].apply(lambda x: ' '.join(x.split(' ')[:1]))
    pop['age'].replace({'80+':'80'},inplace=True)
    pop['age']=pop['age'].astype('int')
    
    pop.drop(['Single Years','Male','Female','Year2','FY'],axis=1,inplace=True)
    
    return pop

pop=get_clean_pop_stack(pop)

In [191]:
def export_pop_data():
    from pandas import ExcelWriter
    with ExcelWriter(output_path+'static_data.xlsx',mode='a') as writer:
        pop.to_excel(writer,sheet_name='pop_age_year')
#export_pop_data()

## Data transformation and initial cleaning 

### Create a new variable aggregating lbw indicators for the new database

In [63]:
lbw_elements = ['105-MA04b2. Deliveries in unit -Live births - less than 2.5kg',
                '105-MA04c2. Deliveries in unit - Fresh still birth - less than 2.5kg',
                '105-MA04d2. Deliveries in unit - Macerated still birth - less than 2.5kg']

def all_nan_sum(x):
    y=x.values
    if np.isnan(y).sum()==len(y):
        x=np.nan
    else:
        x=np.nansum(y)
    return x

In [64]:
lbw_data = new_stack[new_stack['indic'].isin(lbw_elements)].set_index(['district', 
                                                                       'organisationunitid',
                                                                       'indic',
                                                                       'year',
                                                                       'month'])['value'].groupby(['district', 
                                                                                                   'organisationunitid',
                                                                                                   'year',
                                                                                                   'month']).apply(all_nan_sum)
lbw_data=lbw_data.reset_index()
lbw_data['indic']='Babies Born with low birth weight (<2.5Kgs)'
lbw_data

,district,organisationunitid,year,month,value,indic
0,ABIM,JO1cLIghdBv,2020,Apr,0.0,Babies Born with low birth weight (<2.5Kgs)
1,ABIM,JO1cLIghdBv,2020,Feb,NaN,Babies Born with low birth weight (<2.5Kgs)
2,ABIM,JO1cLIghdBv,2020,Jan,0.0,Babies Born with low birth weight (<2.5Kgs)
3,ABIM,JO1cLIghdBv,2020,Mar,0.0,Babies Born with low birth weight (<2.5Kgs)
4,ABIM,JO1cLIghdBv,2020,May,0.0,Babies Born with low birth weight (<2.5Kgs)
...,...,...,...,...,...,...
38215,ZOMBO,xrqX9NsoWWe,2020,Apr,NaN,Babies Born with low birth weight (<2.5Kgs)
38216,ZOMBO,xrqX9NsoWWe,2020,Feb,NaN,Babies Born with low birth weight (<2.5Kgs)
38217,ZOMBO,xrqX9NsoWWe,2020,Jan,NaN,Babies Born with low birth weight (<2.5Kgs)
38218,ZOMBO,xrqX9NsoWWe,2020,Mar,NaN,Babies Born with low birth weight (<2.5Kgs)


In [65]:
new_stack=pd.concat([new_stack,lbw_data])

### Clean a data issue in the old stack regarding reporting rates

In [66]:
old_stack['value'] = np.where(old_stack['indic'].isin(['HMIS 105:1 Actual reports','HMIS 105:1 Expected reports'])
                              & (old_stack['value']==4),1,old_stack['value'])

### Create a new variable aggregating MUAC and W/L SAM and MAM indicators

WIP

# Merge the two

## Select a subset of indicators to merge on

This is done in Excel manually, after running the small function below

In [67]:
def export_var_names():
    from pandas import ExcelWriter
    with ExcelWriter(input_path+'new_old_correspondance.xlsx',mode='a') as writer:
        pd.Series(old_stack['indic'].unique()).to_excel(writer,sheet_name='old_vars')
        pd.Series(new_stack['indic'].unique()).to_excel(writer,sheet_name='new_vars')

#export_var_names()

Here I get back the result into a dict

In [68]:
def replace_var_names():
    from pandas import ExcelFile
    xls = ExcelFile(input_path+'new_old_correspondance.xlsx')
    df = xls.parse(xls.sheet_names[0])
    df.set_index('Old',drop=True,inplace=True)
    old_new_dict=df['New'].to_dict()
    old_stack.replace({'indic': old_new_dict},inplace=True) # Replacing the old names by the new
    target_indics=list(old_new_dict.values()) # Store my target indicators
    return target_indics

In [69]:
target_indics=replace_var_names()

### Check there is no issue with facility ids

In [70]:
# Store the valid ids

old_ids=set(old_stack['organisationunitid'].unique())
new_ids=set(new_stack['organisationunitid'].unique())
valid_ids=list(old_ids.intersection(new_ids))


### Get only the bits of data I am interested in

In [71]:
old_stack_t=old_stack[old_stack['indic'].isin(target_indics) & old_stack['organisationunitid'].isin(valid_ids)].copy()
new_stack_t=new_stack[new_stack['indic'].isin(target_indics) & new_stack['organisationunitid'].isin(valid_ids)].copy()

In [72]:
stack_t=pd.concat([old_stack_t,new_stack_t])
stack_t.reset_index(drop=True,inplace=True)
stack_t.head()

,district,organisationunitid,year,indic,month,value
0,ZOMBO,XikHv88zzDn,2018,EPI - BCG doses given,Apr,7.0
1,ZOMBO,XikHv88zzDn,2018,EPI - BCG doses given,Aug,7.0
2,ZOMBO,XikHv88zzDn,2018,EPI - BCG doses given,Dec,5.0
3,ZOMBO,XikHv88zzDn,2018,EPI - BCG doses given,Feb,NaN
4,ZOMBO,XikHv88zzDn,2018,EPI - BCG doses given,Jan,NaN


### Split reporting vars from others

In [73]:
report_indics=['HMIS 105:01 - OPD Monthly Report (Attendance, Referrals, Conditions,TB, Nutrition) Actual reports 1. National',
               'HMIS 105:01 - OPD Monthly Report (Attendance, Referrals, Conditions,TB, Nutrition) Expected reports 1. National']

In [74]:
stack_t_noreport=stack_t[~stack_t.isin(report_indics)]

In [75]:
stack_t_noreport['indic'].unique()

array(['EPI - BCG doses given', 'EPI - DPT-HepB-HIB 1 doses given',
       'EPI - DPT-HepB-HIB 3 doses given', 'EPI - PCV 1 doses_Under 1',
       'EPI - PCV 3 doses_Under 1', 'EPI - MR 1 doses given',
       '105-MA01. Admissions', '105-MA04a. Deliveries in unit - Total',
       '105-PN01a. Post Natal Attendances - Mother',
       '105-AN01a. ANC 1st Visit for women',
       '105-AN02. ANC 4th Visit for women',
       '105-MA04b1. Deliveries in unit -Live births - Total',
       '105-MA04c1. Deliveries in unit - Fresh still birth - Total',
       '105-MA04d1. Deliveries in unit - Macerated still birth - Total',
       '105-MA11. Newborn deaths (0-7 days)',
       'NUT: Percentage of children/babies born with low birth weight (<2.5kg)',
       '105-NA03a1. Identified malnourished clients(<10) this month - MAM using MUAC',
       '105-NA03e1. Identified malnourished clients(<10) this month - SAM With Oedema',
       '105-NA03c1. Identified malnourished clients(<10) this month - SAM usin

# Flag outliers

## Put our data in the right format 


In [76]:
%%time

def pivot_stack(df):
    pivot_outliers=df.copy().pivot_table(index=['district', 'organisationunitid', 'indic'], columns=['year','month' ]) #,dropna=False)
    pivot_outliers.rename(columns={'value':'with_outiers'},level=0,inplace=True)
    pivot_outliers.columns.rename('type', level=0, inplace=True)
    pivot_outliers.dropna(how='all',axis=0,inplace=True) # looks like there is no all na line to drop
    return pivot_outliers

pivot_outliers=pivot_stack(stack_t_noreport)

Wall time: 4.98 s


## Replace outliers using a std deviation method

In [77]:
# Get the data in the right format

def replace_outliers(pivot_outliers,cutoff):#df
    
    pivot_no_outliers=pd.DataFrame(columns=pivot_outliers.columns,index=pivot_outliers.index)
    pivot_no_outliers.rename(columns={'with_outiers':'without_outliers'},level=0,inplace=True)
    
    for x in pivot_outliers.index: # to exclude
        values = pivot_outliers.loc[x,:].values
        if np.nanstd(values)!=0 and np.isnan(values).sum()!=len(values):
            zscore = abs(stats.zscore(values,nan_policy='omit'))
            new_values = np.where(zscore>cutoff,np.nanmedian(values),values)

        else:
            new_values = values

        pivot_no_outliers.iloc[pivot_outliers.index.get_loc(x),:] = new_values.astype('float')

    return pivot_no_outliers 


In [78]:
%%time
pivot_no_outliers = replace_outliers(pivot_outliers,cutoff=3)


<ipython-input-77-34eb04742e5b>:12: RuntimeWarning: invalid value encountered in greater
  new_values = np.where(zscore>cutoff,np.nanmedian(values),values)


Wall time: 1min 42s


In [79]:
# Get the data in the right format

def replace_outliers_iqr(pivot_outliers,k):#df
    
    pivot_no_outliers=pd.DataFrame(columns=pivot_outliers.columns,index=pivot_outliers.index)
    pivot_no_outliers.rename(columns={'with_outiers':'without_outliers'},level=0,inplace=True)
    
    for x in pivot_outliers.index:
        values = pivot_outliers.loc[x,:].values
        if np.nanstd(values)!=0 and np.isnan(values).sum()!=len(values):
            Q1 = np.nanquantile(values,0.25)
            Q3 = np.nanquantile(values,0.75)
            IQR = Q3 - Q1
            LB = Q1 - k*IQR
            UB = Q3 + k*IQR
            new_values = np.where((values<LB)|(values>UB),np.nanmedian(values),values)

        else:
            new_values = values

        pivot_no_outliers.iloc[pivot_outliers.index.get_loc(x),:] = new_values.astype('float')

    return pivot_no_outliers 

In [80]:
%%time
pivot_no_outliers_iqr = replace_outliers_iqr(pivot_outliers,k=3)

<ipython-input-79-53464bcc185b>:16: RuntimeWarning: invalid value encountered in less
  new_values = np.where((values<LB)|(values>UB),np.nanmedian(values),values)
<ipython-input-79-53464bcc185b>:16: RuntimeWarning: invalid value encountered in greater
  new_values = np.where((values<LB)|(values>UB),np.nanmedian(values),values)


Wall time: 2min 27s


## Stack the outlier corrected data

In [81]:
def pivot_stack(pivot):
    stack = pivot.stack(level=[0,1,2],dropna=False).reset_index()
    stack.rename(columns={0:'value'},inplace=True)
    stack.drop('type',axis=1,inplace=True)
    stack['value']=stack['value'].astype(dtype='float64')
    return stack

In [82]:
stack_t_noout=pivot_stack(pivot_no_outliers)
stack_t_noout_iqr=pivot_stack(pivot_no_outliers_iqr)


## Record which data points were changed

In [83]:
stack_compare = pd.merge(stack_t,stack_t_noout,how='inner',
                           left_on=['district', 'organisationunitid', 'year', 'indic', 'month'],
                           right_on=['district', 'organisationunitid', 'year', 'indic', 'month']).rename(columns={'value_x':'value_out','value_y':'value_noout'})
stack_compare.dropna(subset=['value_out','value_noout'],inplace=True)
stack_compare['changed']=np.where((stack_compare['value_out'] != stack_compare['value_noout']),True,False)
changed = stack_compare[stack_compare['changed']==True]

In [84]:
stack_compare_iqr = pd.merge(stack_t,stack_t_noout_iqr,how='inner',
                           left_on=['district', 'organisationunitid', 'year', 'indic', 'month'],
                           right_on=['district', 'organisationunitid', 'year', 'indic', 'month']).rename(columns={'value_x':'value_out','value_y':'value_noout'})
stack_compare_iqr.dropna(subset=['value_out','value_noout'],inplace=True)
stack_compare_iqr['changed']=np.where((stack_compare_iqr['value_out'] != stack_compare_iqr['value_noout']),True,False)
changed_iqr = stack_compare_iqr[stack_compare_iqr['changed']==True]

In [85]:
changed_iqr

,district,organisationunitid,year,indic,month,value_out,value_noout,changed
41,LUWERO,T4M9UgfqV5q,2019,EPI - BCG doses given,Jul,44.0,22.0,True
394,PADER,auzuV39xOTU,2018,EPI - BCG doses given,Oct,58.0,12.0,True
831,AMURIA,o7CJeTwDapk,2019,EPI - BCG doses given,Feb,20.0,4.0,True
928,MOYO,ZubUtSX9erU,2019,EPI - BCG doses given,Jan,6.0,2.0,True
1010,ABIM,ltIECAx2ppI,2018,EPI - BCG doses given,Dec,57.0,7.0,True
...,...,...,...,...,...,...,...,...
2442625,WAKISO,ziRKF2GJeEU,2020,Babies Born with low birth weight (<2.5Kgs),Feb,12.0,0.0,True
2442686,YUMBE,KOEmIGNgghC,2020,Babies Born with low birth weight (<2.5Kgs),Jan,30.0,2.0,True
2442858,ZOMBO,H9PD4tw7GUE,2020,Babies Born with low birth weight (<2.5Kgs),May,18.0,4.5,True
2442863,ZOMBO,M85HoV8pnUM,2020,Babies Born with low birth weight (<2.5Kgs),May,6.0,1.0,True


In [86]:
changed.to_csv(output_path+'outliers_list.csv')
changed_iqr.to_csv(output_path+'outliers_list_iqr.csv')

# Export this to Tableau

In [87]:
# to check any data point below:

#stack_t[(stack_t['organisationunitid']=='JO1cLIghdBv') & 
        #(stack_t['year']=='2018') & 
        #(stack_t['indic']=='105-NA03e1. Identified malnourished clients(<10) this month - SAM With Oedema') & 
        #(stack_t['month']=='Apr')]['value'].notna()

In [179]:
stack_t['indic'].unique()

array(['EPI - BCG doses given', 'EPI - DPT-HepB-HIB 1 doses given',
       'EPI - DPT-HepB-HIB 3 doses given', 'EPI - PCV 1 doses_Under 1',
       'EPI - PCV 3 doses_Under 1', 'EPI - MR 1 doses given',
       '105-MA01. Admissions', '105-MA04a. Deliveries in unit - Total',
       '105-PN01a. Post Natal Attendances - Mother',
       '105-AN01a. ANC 1st Visit for women',
       '105-AN02. ANC 4th Visit for women',
       '105-MA04b1. Deliveries in unit -Live births - Total',
       '105-MA04c1. Deliveries in unit - Fresh still birth - Total',
       '105-MA04d1. Deliveries in unit - Macerated still birth - Total',
       '105-MA11. Newborn deaths (0-7 days)',
       'NUT: Percentage of children/babies born with low birth weight (<2.5kg)',
       '105-NA03a1. Identified malnourished clients(<10) this month - MAM using MUAC',
       '105-NA03e1. Identified malnourished clients(<10) this month - SAM With Oedema',
       '105-NA03c1. Identified malnourished clients(<10) this month - SAM usin

In [195]:
fac_stack_final = pd.merge(stack_t,stack_t_noout,how='left',
                           left_on=['district', 'organisationunitid', 'year', 'indic', 'month'],
                           right_on=['district', 'organisationunitid', 'year', 'indic', 'month']).rename(columns={'value_x':'value_out','value_y':'value_noout'})

fac_stack_final = pd.merge(fac_stack_final,stack_t_noout_iqr,how='left',
                           left_on=['district', 'organisationunitid', 'year', 'indic', 'month'],
                           right_on=['district', 'organisationunitid', 'year', 'indic', 'month']).rename(columns={'value':'value_noout_iqr'})

fac_stack_final['reported'] = fac_stack_final['value_out'].notna().astype('int')

fac_stack_final['value_noout'] = np.where(fac_stack_final['indic'].isin(report_indics),fac_stack_final['value_out'],fac_stack_final['value_noout'])
fac_stack_final['value_noout_iqr'] = np.where(fac_stack_final['indic'].isin(report_indics),fac_stack_final['value_out'],fac_stack_final['value_noout_iqr'])
fac_stack_final['reported'] = np.where(fac_stack_final['indic'].isin(report_indics),fac_stack_final['value_out'],fac_stack_final['reported'])

fac_pivot_final=fac_stack_final.pivot_table(index=['district','organisationunitid','year','month'], columns=['indic'])

# The pivot creates new NaN values, that need to bereplaced by False for the 'reported' metric

for i in list(fac_pivot_final.columns.levels[1]):
    fac_pivot_final[('reported',i)].replace({np.nan:0},inplace=True)

fac_pivot_final=fac_pivot_final.stack(level=[0])

In [197]:
pivot_export=fac_pivot_final.copy()
pivot_export.to_csv(output_path+'corrected_data_facility.csv')

In [196]:
# Small issue : still some "4" in value_out

fac_stack_final.loc[(fac_stack_final['indic'].isin(report_indics))]

,district,organisationunitid,year,indic,month,value_out,value_noout,value_noout_iqr,reported
3018048,ZOMBO,XikHv88zzDn,2018,"HMIS 105:01 - OPD Monthly Report (Attendance, ...",Jan,1.0,1.0,1.0,1.0
3018049,ZOMBO,XikHv88zzDn,2018,"HMIS 105:01 - OPD Monthly Report (Attendance, ...",Jan,1.0,1.0,1.0,1.0
3018050,ZOMBO,XikHv88zzDn,2018,"HMIS 105:01 - OPD Monthly Report (Attendance, ...",Feb,1.0,1.0,1.0,1.0
3018051,ZOMBO,XikHv88zzDn,2018,"HMIS 105:01 - OPD Monthly Report (Attendance, ...",Feb,1.0,1.0,1.0,1.0
3018052,ZOMBO,XikHv88zzDn,2018,"HMIS 105:01 - OPD Monthly Report (Attendance, ...",Mar,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...
3944035,WAKISO,nYpQROJrK2y,2020,"HMIS 105:01 - OPD Monthly Report (Attendance, ...",Mar,1.0,1.0,1.0,1.0
3944036,WAKISO,nYpQROJrK2y,2020,"HMIS 105:01 - OPD Monthly Report (Attendance, ...",Apr,1.0,1.0,1.0,1.0
3944037,WAKISO,nYpQROJrK2y,2020,"HMIS 105:01 - OPD Monthly Report (Attendance, ...",Apr,1.0,1.0,1.0,1.0
3944038,WAKISO,nYpQROJrK2y,2020,"HMIS 105:01 - OPD Monthly Report (Attendance, ...",May,1.0,1.0,1.0,1.0
